In [2]:
import spacy 
import gzip
import re 
import pandas as pd
from spacy.lang.en import English
from collections import Counter
from itertools import chain
import ahocorasick
import ast
from analyze_m_data import get_occupations_list_en, get_occupations_list
from spacy.lang.de import German
from spacy.lang.it import Italian
from spacy.lang.es import Spanish 
import matplotlib.pyplot as plt
import itertools
from collections import Counter
from operator import itemgetter

In [8]:
filename = "/mount/studenten/projects/caulfiea/cross-verified-database.csv.gz"
csvFilename = gzip.open(filename, 'rb')
df = pd.read_csv(csvFilename, encoding='latin-1')
print(df.head(10))
print(df.columns)

  wikidata_code   birth   death  updated_death_date approx_birth approx_death  \
0      Q1000002  1932.0  1990.0                 NaN          NaN          NaN   
1      Q1000005  1860.0  1927.0                 NaN          NaN          NaN   
2      Q1000006  1971.0     NaN                 NaN          NaN          NaN   
3      Q1000015  1983.0     NaN                 NaN          NaN          NaN   
4      Q1000023  1912.0  1977.0                 NaN          NaN          NaN   
5      Q1000026  1928.0  2016.0                 NaN          NaN          NaN   
6      Q1000034  1818.0  1894.0                 NaN          NaN          NaN   
7      Q1000044  1907.0  1980.0                 NaN          NaN          NaN   
8      Q1000045  1915.0  1987.0                 NaN          NaN        circa   
9      Q1000048  1891.0  1962.0                 NaN          NaN          NaN   

   birth_min  birth_max  death_min  death_max  ... area2_of_rattachment  \
0     1932.0     1932.0     1990.

In [9]:
spanish_female_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/es/es_female_occ_decoded.csv', encoding="utf-8")
es_female_filename = "/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/datasets/final_sets/es_female_dataset.csv"
es_female_df = pd.read_csv(es_female_filename, names= ['wikidata_code', 'title', 'intro', 'gender', 'occupations'],header=0, encoding='utf-8')

In [10]:
es_female_codes = spanish_female_df['wikidata_code'].to_list()
df2 = df.loc[df['wikidata_code'].isin(es_female_codes)]
print(df2.shape[0])

8801


In [11]:
notability_scores = df2[['wikidata_code','number_wiki_editions','sum_visib_ln_5criteria']]
result = pd.merge(spanish_female_df, notability_scores, on='wikidata_code')
print(result.head())

   Unnamed: 0 wikidata_code                    title  \
0           0       Q102213            matilde horne   
1           1     Q10290059          glaura villalba   
2           2     Q10325612  margarida pinto correia   
3           3     Q10346162         patrícia barboza   
4           4     Q10354034         priscilla moreno   

                                               intro  gender  \
0  matilde zagalsky (buenos aires, argentina, 29 ...  female   
1  glaura maria amorim villalba (1978) conocida s...  female   
2  maria margarida amado pinto correia (lisboa, 3...  female   
3  patrícia barboza es una escritora y activista ...  female   
4  cantautora de origen brasileño y panameño (24 ...  female   

                                       occupations  \
0                                   ['translator']   
1                                   ['journalist']   
2  ['journalist', 'television presenter', 'actor']   
3                                       ['writer']   
4       

In [12]:
print(result.columns)
print(result.shape[0])

Index(['Unnamed: 0', 'wikidata_code', 'title', 'intro', 'gender',
       'occupations', 'first_sentence', 'overlapping_occupations',
       'overlapping_occupations_sentence1', 'occupation_in_last_name',
       'number_wiki_editions', 'sum_visib_ln_5criteria'],
      dtype='object')
8801


In [13]:
# Want to get the average of the notability scores of the row in which a certain occ title occurs
# for example, want to compare 'presidente' to 'presidenta' for the spanish female dataset

def contains_profession(row, profession):
    return profession in row

In [ ]:
profession = 'presidenta'
result['overlapping_occupations_sentence1'] = result['overlapping_occupations_sentence1'].apply(ast.literal_eval)
# Filter rows that contain the profession
filtered = result[result['overlapping_occupations_sentence1'].apply(lambda x: contains_profession(x, profession))]

# Calculate the mean score
average_score = filtered['number_wiki_editions'].mean()

print(f"Average score for {profession}: {average_score}")

Average score for presidenta: nan


In [16]:
print(filtered.head())

Empty DataFrame
Columns: [Unnamed: 0, wikidata_code, title, intro, gender, occupations, first_sentence, overlapping_occupations, overlapping_occupations_sentence1, occupation_in_last_name, number_wiki_editions, sum_visib_ln_5criteria]
Index: []


In [ ]:
def get_avg_notability_scores(result):
    all_professions = set()
    for row in result['overlapping_occupations_sentence1']:
        for entry in row:
            all_professions.update(entry.keys())

    # Step 2: Compute average score per profession
    avg_scores = {}
    for profession in all_professions:
        def row_contains_profession(row):
            return any(profession in d for d in row)

        filtered = result[result['overlapping_occupations_sentence1'].apply(row_contains_profession)]
        
        if not filtered.empty:
            avg_scores[profession] = filtered['number_wiki_editions'].mean()
        else:
            avg_scores[profession] = None  # or np.nan if preferred

    # Step 3: Create a nice DataFrame of results
    avg_scores_df = pd.DataFrame.from_dict(avg_scores, orient='index', columns=['Average_Score'])
    print(avg_scores_df)
    return avg_scores

               Average_Score
desaparicion        1.000000
fonoaudiologa       1.000000
poetico             1.000000
vicedirectora       1.000000
alternativas        1.000000
...                      ...
grabador            1.000000
expresidentes       1.000000
portuguesas         1.000000
escritora           1.001229
aparecio            1.000000

[1490 rows x 1 columns]


In [21]:
print(avg_scores['presidenta'])
print(avg_scores['presidente'])
print(avg_scores['actriz'])

1.0
1.0
1.0124254473161034


In [18]:
print(result['overlapping_occupations_sentence1'].head())

0                            [{'traductora': ['Fem']}]
1                                 [{'periodista': []}]
2    [{'actriz': ['Fem']}, {'periodista': []}, {'pr...
3          [{'escritora': ['Fem']}, {'activista': []}]
4                                [{'musica': ['Fem']}]
Name: overlapping_occupations_sentence1, dtype: object
